In [2]:
import pandas as pd
import MeCab
import re

In [3]:
df_file = pd.read_csv('comment.csv')
df_file.head(3)

,videoId,id,textDisplay,authorDisplayName,authorProfileImageUrl,authorChannelUrl,likeCount,publishedAt,updatedAt
0,_TGC3rnWxDc,UgxnHZ9-z6eJOkrn_Ql4AaABAg,「何のための作業なのか」が分かりやすく示されていて理解しやすいです！「こういうものだから」ス...,小波,https://yt3.ggpht.com/iOmVvFfpLpWKuZPKCchCMF1x...,http://www.youtube.com/channel/UCDXgdcGEkte5Q0...,0,2021-11-04T00:05:32Z,2021-11-04T00:05:32Z
1,_TGC3rnWxDc,UgwYcJn_oDBFFzdDi994AaABAg,これ7やなくて8やん,るんるんるん,https://yt3.ggpht.com/ytc/AKedOLQEuDApDnHxrdxh...,http://www.youtube.com/channel/UCx9YZuLiYJ_tx5...,0,2021-10-29T04:27:14Z,2021-10-29T04:27:14Z
2,_TGC3rnWxDc,UgzIMiGtTqa2uxtDcQh4AaABAg,てんちむさんのボケにはファボ１００を与えたい,-ayumu RoundCat,https://yt3.ggpht.com/ytc/AKedOLSDuUlFuQvpVZt_...,http://www.youtube.com/channel/UCwDI_yb6lLf1rP...,0,2021-10-25T21:10:03Z,2021-10-25T21:10:03Z


In [4]:
text = df_file.textDisplay
comment_list = text.tolist()

In [5]:
#単語極性対応表のパス
filepath_emot = 'emotion.txt'

#感情極性表を辞書として登録する
pn_table = pd.read_csv(filepath_emot, engine='python', encoding='utf_8', sep=':', names=('word','reading','POS','PN'))

reading_list = list(pn_table['reading'])
word_list = list(pn_table['word'])
pos_list = list(pn_table['POS'])
pn_list = list(pn_table['PN'])

#単語を評価（感情極性表）を付ける際の辞書
pn_emot = dict(zip(word_list,pn_list))

#tf-idf辞書のパス
filepath_freq = "tf-idf.txt"

#tf-idf辞書を辞書として登録する
pn_csv = pd.read_csv(filepath_freq, engine='python', encoding='utf_8', sep=':', names=('freq','tfidf'))

#単語出現頻度の単語を格納
freq_list = list(pn_csv['freq'])

#TF-IDFの値を格納
tfidf_list = list(pn_csv['tfidf'])

#単語の出現頻度を付加する辞書
pn_freq = dict(zip(freq_list,tfidf_list))

#誹謗中傷ではないと判断された個数
pos_cnt = 0
#誹謗中傷であると判断された個数
neg_cnt = 0
#コメントと点数を格納
com_list = []

In [23]:
def create_score(commet_list):
    for comment in comment_list:
        #分割した単語を格納
        list_com = []
        #分割した単語の代表表記を格納する
        list_repname = []
        #分割した単語の原文化したものを格納
        list_genbun = []
        #分割した単語の品詞を格納
        list_hinsi = []
        #分割した単語の点数を格納
        list_score = []

        tagger = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
        parsed = tagger.parse(comment)
        lines = parsed.split('\n')
        lines = lines[0:-2]
        #print(lines)
        word_list = []
        for line in lines:
            tmp = re.split('\t|,', line)
            #print(tmp)
            if tmp[1] in ["名詞", "動詞", "形容詞"]:
                word_list.append(tmp[7])

        # *を削除する
        words_list = []
        for word in word_list:
            word_result = word.replace('*', '')
            words_list.append(word_result)

        if len(words_list) > 0:
            #単語の判定
            for word in words_list:
                #感情極性辞書の単語と一致
                if word in pn_emot.keys():
                    emot = pn_emot[word]
                    #tf-idf辞書の単語と一致したら感情極性×tf-idfの計算を行う
                    if word in pn_freq:
                        freq = pn_freq[word]
                        list_score.append(emot * freq)
                    else:
                        list_score.append(emot)

            #単語の各点数を合計する
            if len(list_score) > 0:
                score_sum = sum(list_score) / len(list_score)

        # コメントのネガポジ判定
        # if score_sum >= 0:
        #     pos_cnt = pos_cnt + 1
        # else:
        #     neg_cnt = neg_cnt + 1

        com_list.append([comment,score_sum])

    return com_list

In [26]:
result_list = create_score(comment_list)
result_list

[['「何のための作業なのか」が分かりやすく示されていて理解しやすいです！「こういうものだから」スタンスで押し付けられないのがとても嬉しい。',
  -0.2009160880782315],
 ['これ7やなくて8やん', -0.999997],
 ['てんちむさんのボケにはファボ１００を与えたい', -0.3044430509615885],
 ['18:00付近\u3000右下の板書違う', -0.2873032607826967],
 ['「何のための作業なのか」が分かりやすく示されていて理解しやすいです！「こういうものだから」スタンスで押し付けられないのがとても嬉しい。',
  -0.2009160880782315],
 ['これ7やなくて8やん', -0.999997],
 ['てんちむさんのボケにはファボ１００を与えたい', -0.3044430509615885],
 ['18:00付近\u3000右下の板書違う', -0.2873032607826967],
 ['「何のための作業なのか」が分かりやすく示されていて理解しやすいです！「こういうものだから」スタンスで押し付けられないのがとても嬉しい。',
  -0.2009160880782315],
 ['これ7やなくて8やん', -0.999997],
 ['てんちむさんのボケにはファボ１００を与えたい', -0.3044430509615885],
 ['18:00付近\u3000右下の板書違う', -0.2873032607826967],
 ['Thanks!', -0.2873032607826967],
 ['もうじき還暦の算数好きのオッさんです。カミカミするたび、ほっぺをペチン。心配しながらありがたく拝聴しております。がんばれ〜',
  -0.11875133507350517],
 ['españa', -0.11875133507350517],
 ['Зачем это мне', -0.11875133507350517],
 ['てんちむワロタ', -0.11875133507350517],
 ['1:55何気にクラメルの公式が成り立っていることを実感できる瞬間www', -0.12972966684684265],
 ['同じ内容の授業でも、１００分かける教授と２０分で

In [29]:
df = pd.DataFrame(result_list, columns=['comment', 'score'])
df

,comment,score
0,「何のための作業なのか」が分かりやすく示されていて理解しやすいです！「こういうものだから」ス...,-0.200916
1,これ7やなくて8やん,-0.999997
2,てんちむさんのボケにはファボ１００を与えたい,-0.304443
3,18:00付近 右下の板書違う,-0.287303
4,「何のための作業なのか」が分かりやすく示されていて理解しやすいです！「こういうものだから」ス...,-0.200916
...,...,...
1857,区分求積の動画出すって言ったの覚えてっか！？,-0.413049
1858,１解析〜！２解析〜！ベクトル解析〜!,-0.004232
1859,次回のテーマ余因子展開はよびのりの最初の動画と同じですね原点回帰ですね,-0.108204
1860,予備校のノリで学ぶ「大学の数学・物理」 学校で行列やったよー,-0.223723


In [36]:
# CSVファイルとして出力
df.to_csv("comment_score.csv")

In [40]:
df[df.score > 0]
# CSVファイルとして出力
df[df.score > 0].to_csv("comment_pos.csv")

In [41]:
df[df.score > 0]

,comment,score
24,てんちむの豊胸でわらったわ,0.028260
28,たくみさんボケめちゃおもしろいけど人を笑かす間じゃない笑言い方変えれば松本人志なれるのに,0.248208
56,動画1.5倍速になっててショートコントの最後を「俺お前と結婚するわ」って聞き間違えたから概要...,0.068315
63,行列式の一般形見つけた人マジで天才だと思う,0.329695
90,二次形式あれば助かります🙇,0.011030
...,...,...
1817,誰か30言っちゃいけないゲームのカラクリを教えて下さい🙏,0.219112
1818,@次郎玉ねぎ ありがとうございます！今度から使ってみますね😀,0.001033
1819,@r y なるほど！ありがとうございますm(__)m,0.001033
1831,好き,0.932040


In [33]:
score_list = df['score']
for score in score_list:
# コメントのネガポジ判定
    if score >= 0:
        pos_cnt = pos_cnt + 1
    else:
        neg_cnt = neg_cnt + 1

In [35]:
print(pos_cnt)
print(neg_cnt)

144
1718
